#2025_07_02 FLUTTER 실험용 DETECTION 모델

- IMGSIZE: 640
- NUMCLASS: 10[링크 텍스트](https://)

In [ ]:
import os
import cv2
import json

# 경로 설정
BASE_DIR = "/content/0617_YOLO"
OUTPUT_DIR = "/content/YOLO_output"
TARGET_SIZE = 640
CATEGORY_ID_TO_CLASS = {1: 0}  # 'pill': 0

# 출력 폴더 생성
for split in ['train', 'val']:
    os.makedirs(os.path.join(OUTPUT_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, 'labels', split), exist_ok=True)

# 이미지 리사이즈 함수
def resize_with_padding(image, target_size):
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h))
    padded = cv2.copyMakeBorder(
        resized,
        (target_size - new_h) // 2, (target_size - new_h + 1) // 2,
        (target_size - new_w) // 2, (target_size - new_w + 1) // 2,
        borderType=cv2.BORDER_CONSTANT, value=(255, 255, 255)
    )
    return padded, scale, (target_size - new_w) // 2, (target_size - new_h) // 2

# JSON → YOLO TXT 변환
def convert_json_to_yolo(json_path, scale, pad_w, pad_h):
    with open(json_path, 'r') as f:
        data = json.load(f)

    yolo_lines = []
    for ann in data.get("annotations", []):
        category_id = ann.get("category_id")
        if category_id not in CATEGORY_ID_TO_CLASS:
            continue  # 정의되지 않은 카테고리는 건너뜀

        class_id = CATEGORY_ID_TO_CLASS[category_id]
        x, y, w, h = ann["bbox"]

        x1 = x * scale + pad_w
        y1 = y * scale + pad_h
        w_scaled = w * scale
        h_scaled = h * scale

        xc = (x1 + w_scaled / 2) / TARGET_SIZE
        yc = (y1 + h_scaled / 2) / TARGET_SIZE
        norm_w = w_scaled / TARGET_SIZE
        norm_h = h_scaled / TARGET_SIZE

        yolo_lines.append(f"{class_id} {xc:.6f} {yc:.6f} {norm_w:.6f} {norm_h:.6f}")
    return yolo_lines

# 변환 루프 실행
for split in ["train", "val"]:
    img_dir = os.path.join(BASE_DIR, "images", split)
    label_dir = os.path.join(BASE_DIR, "labels", f"{split}_json")
    out_img_dir = os.path.join(OUTPUT_DIR, "images", split)
    out_lbl_dir = os.path.join(OUTPUT_DIR, "labels", split)

    for img_name in os.listdir(img_dir):
        if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            continue
        base = os.path.splitext(img_name)[0]
        img_path = os.path.join(img_dir, img_name)
        json_path = os.path.join(label_dir, base + ".json")
        image = cv2.imread(img_path)
        if image is None or not os.path.exists(json_path):
            continue
        resized_img, scale, pad_w, pad_h = resize_with_padding(image, TARGET_SIZE)
        cv2.imwrite(os.path.join(out_img_dir, img_name), resized_img)
        yolo_lines = convert_json_to_yolo(json_path, scale, pad_w, pad_h)
        with open(os.path.join(out_lbl_dir, base + ".txt"), "w") as f:
            f.write("\n".join(yolo_lines))


In [ ]:
# 저장 디렉토리
yaml_dir = "/content/YOLO_output"
yaml_path = os.path.join(yaml_dir, "data.yaml")

# 디렉토리 없으면 생성
os.makedirs(yaml_dir, exist_ok=True)

# YAML 내용 작성
yaml_content = """train: /content/YOLO_output/images/train
val: /content/YOLO_output/images/val

nc: 1
names: ['pill']
"""

# 파일 저장
with open(yaml_path, "w") as f:
    f.write(yaml_content)

In [ ]:
!pip install ultralytics

In [ ]:
!yolo detect train \
  model=yolov8n.pt \
  data=/content/YOLO_output/data.yaml \
  imgsz=640 \
  epochs=50 \
  batch=8 \
  project=runs/detect \
  name=exp_yolo8 \
  seed=42 \
  patience=5


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 6.25M/6.25M [00:00<00:00, 303MB/s]
Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/YOLO_output/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, 

In [ ]:
!yolo detect predict \
  model=/content/runs/detect/exp_yolo8/weights/best.pt \
  source=/content/testimg \
  imgsz=640 \
  conf=0.25 \
  save=True \
  save_txt=True \
  save_conf=True \
  project=runs/detect \
  name=pred_yolo8 \
  exist_ok=True


Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

image 1/7 /content/testimg/K-000250-000573-002483-023223_0_2_0_2_70_000_200.png: 640x512 3 pills, 44.6ms
image 2/7 /content/testimg/K-000250-000573-023223-044834_0_2_0_2_90_000_200.png: 640x512 4 pills, 7.5ms
image 3/7 /content/testimg/K-000250-002483-012778-022362_0_2_0_2_75_000_200.png: 640x512 2 pills, 8.2ms
image 4/7 /content/testimg/K-000250-012778-013395-037777_0_2_0_2_70_000_200.png: 640x512 3 pills, 7.5ms
image 5/7 /content/testimg/K-000573-005094-006192-044834_0_2_0_2_75_000_200.png: 640x512 4 pills, 9.1ms
image 6/7 /content/testimg/K-003351-016262-019232_0_2_0_2_75_000_200.png: 640x512 3 pills, 7.1ms
image 7/7 /content/testimg/K-003351-018147-038162_0_2_0_2_70_000_200.png: 640x512 3 pills, 7.2ms
Speed: 2.9ms preprocess, 13.0ms inference, 21.7ms postprocess per image at shape (1, 3, 640, 512)
Results sa